In [ ]:
from deepface import DeepFace
import cv2
import os
import datetime
import platform

# Kiểm tra xem có thể hiển thị GUI không (dùng cho Jupyter hoặc hệ thống không có màn hình)
def is_gui_available():
    return platform.system() != "Linux" or os.environ.get("DISPLAY") is not None

# -----------------------------------------------
# Hàm phân tích DeepFace trên một frame ảnh
# -----------------------------------------------
def analyze_and_draw(frame):
    try:
        results = DeepFace.analyze(frame, actions=["age", "gender", "emotion"], enforce_detection=False)
        for face in results:
            region = face.get("region", {})
            if not region: continue
            x, y, w, h = region.values()

            gender = face.get("gender", "?")
            age = face.get("age", "?")
            emotion = face.get("dominant_emotion", "?")

            # Vẽ khung
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            label = f"{gender}, {age}, {emotion}"
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    except Exception as e:
        print("⚠️ Lỗi phân tích gương mặt:", e)
    return frame

# Tạo thư mục lưu kết quả
def create_output_dir():
    os.makedirs("result_deepface", exist_ok=True)
    return "result_deepface"

# -----------------------------------------------
# 1. Phân tích từ ảnh
# -----------------------------------------------
def analyze_from_image(image_path):
    if not os.path.exists(image_path):
        print(" Không tìm thấy ảnh:", image_path)
        return

    image = cv2.imread(image_path)
    result = analyze_and_draw(image)

    if is_gui_available():
        cv2.imshow("Result from image", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    output_dir = create_output_dir()
    output_path = os.path.join(output_dir, f"deepface_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, result)
    print(f"💾 Ảnh kết quả đã lưu tại: {output_path}")

# -----------------------------------------------
# 2. Phân tích từ video
# -----------------------------------------------
def analyze_from_video(video_path):
    if not os.path.exists(video_path):
        print(" Không tìm thấy video:", video_path)
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(" Không mở được video.")
        return

    output_dir = create_output_dir()
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        result = analyze_and_draw(frame)

        if is_gui_available():
            cv2.imshow("DeepFace video", result)

        # Lưu từng frame
        filename = f"{output_dir}/deepface_frame_{frame_id}.jpg"
        cv2.imwrite(filename, result)

        if is_gui_available() and cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()
    print(" Đã kết thúc phân tích video.")

# -----------------------------------------------
# 3. Phân tích từ webcam
# -----------------------------------------------
def analyze_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print(" Không thể mở webcam.")
        return

    output_dir = create_output_dir()
    frame_id = 0
    saved_count = 0

    print("📡 Đang phân tích từ webcam... Nhấn 'e' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id += 1
        try:
            results = DeepFace.analyze(frame, actions=["age", "gender", "emotion"], enforce_detection=False)
            if results:
                for face in results:
                    region = face.get("region", {})
                    if not region: continue
                    x, y, w, h = region.values()
                    gender = face.get("gender", "?")
                    age = face.get("age", "?")
                    emotion = face.get("dominant_emotion", "?")

                    # Vẽ khung và label
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    label = f"{gender}, {age}, {emotion}"
                    cv2.putText(frame, label, (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

                # Lưu ảnh có khuôn mặt
                saved_count += 1
                filename = f"{output_dir}/deepface_webcam_{frame_id}.jpg"
                cv2.imwrite(filename, frame)
                print(f"💾 Đã lưu ảnh có khuôn mặt: {filename}")
        except Exception as e:
            print(" Lỗi phân tích:", e)

        if is_gui_available():
            cv2.imshow("DeepFace webcam", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('e') or key == 27:
            print(" Đã thoát webcam.")
            break

    cap.release()
    if is_gui_available():
        cv2.destroyAllWindows()

    print(f" Tổng số ảnh chứa khuôn mặt đã lưu: {saved_count}")


# -----------------------------------------------
# Menu chính
# -----------------------------------------------
def main():
    while True:
        print("\n=== DeepFace Face Detection ===")
        print("1. Analyze from image")
        print("2. Analyze from video")
        print("3. Analyze from webcam")
        print("4. Exit")
        choice = input("👉 Nhập lựa chọn (1/2/3/4): ")

        if choice == '1':
            path = input("🖼️ Nhập đường dẫn ảnh: ")
            analyze_from_image(path)
        elif choice == '2':
            path = input("🎞️ Nhập đường dẫn video: ")
            analyze_from_video(path)
        elif choice == '3':
            analyze_from_webcam()
        elif choice == '4':
            print("👋 Kết thúc chương trình.")
            break
        else:
            print("⚠️ Lựa chọn không hợp lệ.")

if __name__ == "__main__":
    main()



25-07-14 16:04:42 - Directory C:\Users\ACER\.deepface has been created
25-07-14 16:04:42 - Directory C:\Users\ACER\.deepface\weights has been created

=== DeepFace Face Detection ===
1. Analyze from image
2. Analyze from video
3. Analyze from webcam
4. Exit
👋 Kết thúc chương trình.
